# 🚀 NASA GCN Data Pipeline

## Visão Geral
Pipeline **Delta Live Tables (DLT)** para ingestão de eventos astronômicos da NASA Gamma-ray Coordinates Network (GCN).

### O que é o NASA GCN?
O [GCN (Gamma-ray Coordinates Network)](https://gcn.nasa.gov/) é um sistema de alerta em tempo real que distribui informações sobre:
- 🌟 **Gamma-ray Bursts (GRBs)** - Explosões de raios gama
- 🌊 **Ondas Gravitacionais (IGWN)** - Detecções do LIGO/Virgo
- ☄️ **Eventos de Neutrinos** - IceCube e outros detectores
- 🔭 **Transientes Astronômicos** - Supernovas, kilonovas, etc.

---

## Arquitetura Medallion (Bronze → Silver → Gold)

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│     BRONZE      │    │     SILVER      │    │      GOLD       │
│   (Raw Data)    │───▶│  (Parsed Data)  │───▶│  (Aggregated)   │
└─────────────────┘    └─────────────────┘    └─────────────────┘
     gcn_raw           gcn_classic_text            (futuro)
                       gcn_classic_voevent
                       gcn_classic_binary
                       gcn_notices
                       gcn_circulars
                       igwn_gwalert
                       gcn_heartbeat
```

### Camadas:
| Camada | Propósito | Tabelas |
|--------|-----------|----------|
| **Bronze** | Dados brutos exatamente como chegaram do Kafka | `gcn_raw` |
| **Silver** | Dados filtrados por tipo, decodificados, com schema definido | 7 tabelas por categoria |
| **Gold** | Agregações, métricas, dashboards (implementação futura) | - |

## Configuração e Importações

### Dependências:
- **`dlt`**: Módulo Delta Live Tables do Databricks
- **`nasa_gcn.config`**: Configurações Kafka/OAuth (definido em `src/nasa_gcn/config.py`)

### Variáveis de Ambiente:
As credenciais são injetadas via `spark.conf` pelo pipeline DLT, definidas em `nasa_gcn.pipeline.yml`.

In [ ]:
# ==============================================================================
# IMPORTS E CONFIGURAÇÃO
# ==============================================================================

import dlt  # Delta Live Tables - framework declarativo para pipelines
import sys

# Adiciona o path do bundle para importar módulos locais
# bundle.sourcePath é definido em nasa_gcn.pipeline.yml → configuration
sys.path.append(spark.conf.get("bundle.sourcePath", "."))

# PySpark functions para transformações
from pyspark.sql.functions import col, decode, split, current_timestamp

# Configurações Kafka do módulo nasa_gcn
# get_kafka_options() retorna dict com bootstrap.servers, OAuth, etc.
from nasa_gcn.config import get_kafka_options

---

## 🥉 Bronze Layer: Ingestão Raw

### Tabela: `gcn_raw`

**Propósito**: Captura TODAS as mensagens Kafka do GCN, exatamente como chegam, sem transformação.

**Por que manter dados raw?**
- 📜 **Auditoria**: Registro completo do que foi recebido
- 🔄 **Reprocessamento**: Se o parsing Silver falhar, temos a fonte original
- 🔍 **Debug**: Facilita troubleshooting de problemas de schema

**Colunas:**
| Coluna | Tipo | Descrição |
|--------|------|------------|
| `message_key` | STRING | Chave da mensagem Kafka (pode ser nula) |
| `value` | BINARY | Conteúdo raw da mensagem (bytes) |
| `topic` | STRING | Tópico Kafka de origem (ex: `gcn.classic.text.FERMI_GBM_FLT_INTERNAL`) |
| `partition` | INT | Partição Kafka |
| `offset` | LONG | Offset da mensagem na partição |
| `kafka_timestamp` | TIMESTAMP | Timestamp atribuído pelo Kafka |
| `ingestion_timestamp` | TIMESTAMP | Quando o Databricks recebeu a mensagem |

In [ ]:
# ==============================================================================
# BRONZE: gcn_raw - Todos os eventos GCN em formato bruto
# ==============================================================================

@dlt.table(
    name="gcn_raw",
    comment="Raw NASA GCN Kafka messages (all topics) - Bronze layer",
    table_properties={"quality": "bronze"}
)
def gcn_raw():
    """
    Ingestão de mensagens raw do NASA GCN Kafka.
    
    Conecta-se aos tópicos GCN usando OAuth e armazena todas as mensagens
    sem transformação. Os tópicos são definidos em config.py via subscribePattern.
    
    Returns:
        DataFrame: Stream de mensagens Kafka com metadados
    """
    # Obtém configurações Kafka (bootstrap, OAuth, tópicos)
    kafka_options = get_kafka_options()
    
    return (
        spark.readStream
        .format("kafka")              # Conector Kafka estruturado
        .options(**kafka_options)      # Credenciais e configurações
        .load()
        .select(
            # Converte key de bytes para string (geralmente é null no GCN)
            col("key").cast("string").alias("message_key"),
            col("value"),              # Mantém como bytes (BINARY)
            col("topic"),              # Tópico de origem
            col("partition"),          # Partição Kafka
            col("offset"),             # Offset para tracking
            col("timestamp").alias("kafka_timestamp"),
            current_timestamp().alias("ingestion_timestamp")  # Quando ingerimos
        )
    )

---

## 🥈 Silver Layer: Tabelas Multiplexadas por Categoria

A camada Silver filtra e transforma os dados raw por tipo de evento.
Cada tabela lê do `gcn_raw` e aplica filtros e decodificação específicos.

### Por que multiplex?
O GCN envia diversos tipos de mensagens em formatos diferentes:
- **Text**: Alertas em texto simples (formato legado)
- **VoEvent**: XML estruturado (padrão IVOA)
- **Binary**: Dados binários compactos
- **JSON**: Novo formato estruturado (gcn.notices.*)
- **Circulars**: Relatórios de astrônomos em formato JSON

### Tabela: `gcn_classic_text`

**Tópicos**: `gcn.classic.text.*` (ex: FERMI_GBM, SWIFT_BAT, etc.)

**Formato**: Texto simples ASCII com campos chave=valor.

Exemplo de mensagem:
```
TITLE:          GCN/FERMI NOTICE
NOTICE_DATE:    Mon 01 Jan 24 12:34:56 UT
TRIGGER_NUM:    123456789
...
```

In [ ]:
# ==============================================================================
# SILVER: gcn_classic_text - Alertas em formato texto
# ==============================================================================

@dlt.table(
    name="gcn_classic_text",
    comment="Classic text format GCN alerts (FERMI, SWIFT, etc.) - Silver layer",
    table_properties={"quality": "silver"}
)
def gcn_classic_text():
    """
    Filtra mensagens de tópicos gcn.classic.text.* e decodifica para UTF-8.
    
    O campo event_type é extraído do nome do tópico:
    gcn.classic.text.FERMI_GBM_FLT → event_type = "FERMI_GBM_FLT"
    """
    return (
        dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.classic.text."))
        .select(
            col("message_key"),
            # Decodifica bytes para string UTF-8
            decode(col("value"), "UTF-8").alias("message_text"),
            col("topic"),
            # Extrai tipo de evento: split por '.' e pega índice 3
            split(col("topic"), r"\.").getItem(3).alias("event_type"),
            col("kafka_timestamp"),
            col("ingestion_timestamp"),
            col("partition"),
            col("offset")
        )
    )

### Tabela: `gcn_classic_voevent`

**Tópicos**: `gcn.classic.voevent.*`

**Formato**: XML seguindo o padrão [VOEvent](https://www.ivoa.net/documents/VOEvent/) da IVOA.

VOEvent é um formato rico que inclui:
- Coordenadas celestes (RA, Dec)
- Incertezas e regiões de erro
- Parâmetros científicos do evento

In [ ]:
# ==============================================================================
# SILVER: gcn_classic_voevent - Alertas em formato XML VOEvent
# ==============================================================================

@dlt.table(
    name="gcn_classic_voevent",
    comment="Classic VoEvent XML format GCN alerts - Silver layer",
    table_properties={"quality": "silver"}
)
def gcn_classic_voevent():
    """
    Filtra mensagens VoEvent (XML) e decodifica para UTF-8.
    
    O XML pode ser parseado posteriormente usando from_xml() ou
    bibliotecas Python como lxml.
    """
    return (
        dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.classic.voevent."))
        .select(
            col("message_key"),
            decode(col("value"), "UTF-8").alias("voevent_xml"),
            col("topic"),
            split(col("topic"), r"\.").getItem(3).alias("event_type"),
            col("kafka_timestamp"),
            col("ingestion_timestamp"),
            col("partition"),
            col("offset")
        )
    )

### Tabela: `gcn_classic_binary`

**Tópicos**: `gcn.classic.binary.*`

**Formato**: Pacotes binários de 160 bytes (formato legado BACODINE/GCN).

⚠️ **Nota**: Dados binários são mantidos como `BINARY` para posterior decodificação com parsers específicos.

In [ ]:
# ==============================================================================
# SILVER: gcn_classic_binary - Alertas em formato binário
# ==============================================================================

@dlt.table(
    name="gcn_classic_binary",
    comment="Classic binary format GCN alerts (160-byte packets) - Silver layer",
    table_properties={"quality": "silver"}
)
def gcn_classic_binary():
    """
    Filtra mensagens binárias (mantém como bytes).
    
    O formato binário requer decodificação especial usando a especificação
    BACODINE. Considere usar gcn-kafka Python library para parsing.
    """
    return (
        dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.classic.binary."))
        .select(
            col("message_key"),
            col("value").alias("binary_data"),  # Mantém como BINARY
            col("topic"),
            split(col("topic"), r"\.").getItem(3).alias("event_type"),
            col("kafka_timestamp"),
            col("ingestion_timestamp"),
            col("partition"),
            col("offset")
        )
    )

### Tabela: `gcn_notices`

**Tópicos**: `gcn.notices.*` (ex: `gcn.notices.swift.bat.guano`)

**Formato**: JSON estruturado - o formato mais moderno e fácil de trabalhar.

Schema unificado documentado em: https://gcn.nasa.gov/docs/notices/schema

In [ ]:
# ==============================================================================
# SILVER: gcn_notices - Alertas no novo formato JSON
# ==============================================================================

@dlt.table(
    name="gcn_notices",
    comment="New JSON format GCN notices (unified schema) - Silver layer",
    table_properties={"quality": "silver"}
)
def gcn_notices():
    """
    Filtra notices JSON (formato novo) e decodifica para UTF-8.
    
    Este é o formato recomendado para novos consumidores.
    O JSON pode ser parseado com from_json() usando o schema unificado GCN.
    
    Estrutura do tópico: gcn.notices.{mission}.{type}
    Exemplo: gcn.notices.swift.bat.guano → mission = "swift"
    """
    return (
        dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.notices."))
        .select(
            col("message_key"),
            decode(col("value"), "UTF-8").alias("notice_json"),
            col("topic"),
            # Extrai missão: gcn.notices.{mission}.* → índice 2
            split(col("topic"), r"\.").getItem(2).alias("mission"),
            col("kafka_timestamp"),
            col("ingestion_timestamp"),
            col("partition"),
            col("offset")
        )
    )

### Tabela: `gcn_circulars`

**Tópico**: `gcn.circulars`

**O que são Circulars?**

GCN Circulars são relatórios breves escritos por astrônomos sobre:
- Descobertas de novos transientes
- Observações de follow-up
- Interpretações científicas

São como "tweets científicos" da comunidade astronômica!

In [ ]:
# ==============================================================================
# SILVER: gcn_circulars - Relatórios de astrônomos
# ==============================================================================

@dlt.table(
    name="gcn_circulars",
    comment="GCN Circulars - astronomer reports and observations - Silver layer",
    table_properties={"quality": "silver"}
)
def gcn_circulars():
    """
    Filtra GCN Circulars (relatórios de astrônomos).
    
    Circulars contêm metadados (subject, submitter, date) e o texto
    do relatório em formato livre.
    
    Exemplo de uso: análise de sentimento, extração de coordenadas,
    correlação com eventos GCN Notices.
    """
    return (
        dlt.read_stream("gcn_raw")
        .filter(col("topic") == "gcn.circulars")
        .select(
            col("message_key"),
            decode(col("value"), "UTF-8").alias("circular_json"),
            col("topic"),
            col("kafka_timestamp"),
            col("ingestion_timestamp"),
            col("partition"),
            col("offset")
        )
    )

### Tabela: `igwn_gwalert`

**Tópico**: `igwn.gwalert`

**🌊 Ondas Gravitacionais!**

Alertas do IGWN (International Gravitational-Wave Observatory Network):
- LIGO (EUA)
- Virgo (Europa)
- KAGRA (Japão)

Estes são eventos raros e cientificamente significativos!

In [ ]:
# ==============================================================================
# SILVER: igwn_gwalert - Alertas de Ondas Gravitacionais
# ==============================================================================

@dlt.table(
    name="igwn_gwalert",
    comment="IGWN Gravitational Wave Alerts (LIGO/Virgo/KAGRA) - Silver layer",
    table_properties={"quality": "silver"}
)
def igwn_gwalert():
    """
    Filtra alertas de ondas gravitacionais do IGWN.
    
    Eventos incluem:
    - Preliminary: Detecção inicial, alta incerteza
    - Initial: Análise rápida confirmada
    - Update: Refinamentos de parâmetros
    - Retraction: Evento descartado como ruído
    
    Priorize eventos com classification "BNS" (Binary Neutron Star)
    ou "NSBH" (Neutron Star-Black Hole) para multi-messenger astronomy!
    """
    return (
        dlt.read_stream("gcn_raw")
        .filter(col("topic") == "igwn.gwalert")
        .select(
            col("message_key"),
            decode(col("value"), "UTF-8").alias("gwalert_json"),
            col("topic"),
            col("kafka_timestamp"),
            col("ingestion_timestamp"),
            col("partition"),
            col("offset")
        )
    )

### Tabela: `gcn_heartbeat`

**Tópico**: `gcn.heartbeat`

**💓 Pulso de Vida do Sistema**

Mensagens de heartbeat são enviadas periodicamente para:
- 🔍 Verificar conectividade com o broker Kafka
- 📊 Monitorar latência do pipeline
- 🚨 Detectar quedas de conexão

In [ ]:
# ==============================================================================
# SILVER: gcn_heartbeat - Mensagens de monitoramento
# ==============================================================================

@dlt.table(
    name="gcn_heartbeat",
    comment="GCN Heartbeat/test messages for monitoring - Silver layer",
    table_properties={"quality": "silver"}
)
def gcn_heartbeat():
    """
    Filtra mensagens de heartbeat (pulso do sistema).
    
    Útil para:
    - Criar alertas quando heartbeats param de chegar
    - Medir latência end-to-end (timestamp GCN vs ingestion_timestamp)
    - Validar que o pipeline está funcionando sem eventos reais
    
    Frequência esperada: ~1 mensagem a cada poucos segundos
    """
    return (
        dlt.read_stream("gcn_raw")
        .filter(col("topic") == "gcn.heartbeat")
        .select(
            col("message_key"),
            decode(col("value"), "UTF-8").alias("heartbeat_json"),
            col("topic"),
            col("kafka_timestamp"),
            col("ingestion_timestamp"),
            col("partition"),
            col("offset")
        )
    )